## TODO: 
- Build Optimizer Agent

---

# AAI-520-02 Final Team Project: Multi-Agent System

#### Contributors:
- Alexander J Padin
- Thomas Geraci
- Ali Mohtat

## Imports

In [5]:
import os
from typing import Dict, Any, List
from dotenv import load_dotenv
import pandas as pd
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool 
from typing import Dict, Any
from IPython.display import display, Markdown
from pathlib import Path

load_dotenv() # load .env variable with API Keys (see .env-example)

OUT_DIR  = Path("outputs"); OUT_DIR.mkdir(exist_ok=True)
MEM_DIR  = Path("memory"); MEM_DIR.mkdir(exist_ok=True)

## Large Language Models (LLMs)  
In our system, LLMs are the foundational models that power each agent's reasoning, text generation, and decision-making. We explicitly define which LLM each agent will use (e.g. OpenAI’s GPT models or Anthropic’s Claude) so the agents operate with predictable performance and behavior.

In [7]:
claude_llm = LLM(
    model="claude-3-5-sonnet-20240620",
    base_url="https://api.anthropic.com",
    api_key=os.environ["ANTHROPIC_API_KEY"]
)

openai_llm = LLM(
    model="openai/gpt-4o",
    api_key=os.environ["OPENAI_API_KEY"]
)

## Knowledge Base Helper Functions
This cell defines helper functions that act as a lightweight knowledge base. The function _kb_path creates a path inside the memory directory for a given ticker symbol. The function kb_append_lesson adds a JSON line entry containing the lesson text and a timestamp. The function kb_load_lessons reads all of the saved JSON lines and returns them as a list of dictionaries. Together these functions provide a simple way to store and recall one sentence lessons from previous reviews so the agent can build and reference long term memory over multiple sessions.

In [9]:
#Knowledge Base helpers
def _kb_path(ticker: str) -> Path:
    p = MEM_DIR / f"kb_{ticker.upper()}.jsonl"
    p.touch(exist_ok=True)
    return p

def kb_append_lesson(ticker: str, lesson: dict) -> None:
    p = _kb_path(ticker)
    with p.open("a", encoding="utf-8") as f:
        f.write(json.dumps(lesson, ensure_ascii=False) + "\n")

def kb_load_lessons(ticker: str) -> list[dict]:
    p = _kb_path(ticker)
    out = []
    with p.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                out.append(json.loads(line))
    return out

## Tools
Tools are modular, executable functions that agents can call to perform tasks (e.g., fetching data or interacting with APIs). In CrewAI, we wrap a function with `@tool("Tool Name")`, which registers it with metadata. The agent can then decide when to call which tool and how to use its output.  

### Tools in this project  
- `yf_prices(ticker, period='90d', interval='1d')`: fetch recent OHLCV (Open-High-Low-Close-Volume) price data  
- `yf_fundamentals(ticker)`: retrieve a compact fundamentals snapshot (e.g. market cap, P/E, EPS)  
- `yf_dividends(ticker, limit=5)`: fetch recent dividend events (up to N)  
- `yf_calendar(ticker)`: get upcoming or recent calendar events (e.g. earnings)  

### OHLCV Fetcher
This tool fetches recent OHLCV (Open, High, Low, Close, Volume) data for a given stock ticker over a specified time period (90 days) using `yfinance`. It returns up to the last 10 rows of data (the “tail”) plus metadata indicating how many rows were available. If no data is found, it returns an empty result with zero rows.

In [12]:
@tool("Fetch recent OHLCV prices from Yahoo Finance")
def yf_prices(ticker: str, period: str = "90d", interval: str = "1d"):
    """Returns recent OHLCV prices from Yahoo Finance""" # need docstring for tools
    
    tk = yf.Ticker(ticker)
    df = tk.history(period=period, interval=interval, auto_adjust=False)
    
    if df is None or df.empty: # if empty
        return {"ticker": ticker, "period": period, "interval": interval, "rows": 0, "tail10": []}
        
    out = df.reset_index().tail(10).to_dict(orient="records")
    return {"ticker": ticker, "period": period, "interval": interval, "rows": len(df), "tail10": out}

### Fundamentals Snapshot Fetcher
This tool fetches a compact fundamentals snapshot for a given ticker using `yfinance`. It tries to use `fast_info` (a lightweight subset of metadata) when available; if not, it falls back to `Ticker.info`. The returned dict includes fields like last price, market cap, P/E, EPS, 52-week high/low, and others if available.

In [14]:
@tool("Fetch a fundamentals snapshot from Yahoo Finance")
def yf_fundamentals(ticker: str) -> Dict[str, Any]:
    """Returns a fundamentals snapshot from Yahoo Finance"""
    
    tk = yf.Ticker(ticker)
    info = getattr(tk, "fast_info", None)
    
    if info:
        # if fast_info exists, use selected attributes
        attrs = [
            "last_price", "market_cap", "currency", 
            "year_high", "year_low", "shares_outstanding"
        ]
        fi = {k: getattr(info, k, None) for k in attrs}
        
    else: # else, get data form Ticker.info
        try:
            raw = tk.info
        except Exception:
            raw = {}
        fi = {
            "last_price": raw.get("currentPrice"),
            "market_cap": raw.get("marketCap"),
            "currency": raw.get("currency"),
            "year_high": raw.get("fiftyTwoWeekHigh"),
            "year_low": raw.get("fiftyTwoWeekLow"),
            "shares_outstanding": raw.get("sharesOutstanding"),
            "trailing_pe": raw.get("trailingPE"),
            "forward_pe": raw.get("forwardPE"),
            "eps": raw.get("trailingEps") or raw.get("epsTrailingTwelveMonths"),
            "short_name": raw.get("shortName"),
            "beta": raw.get("beta"),
        }
        
    return {"ticker": ticker, "fundamentals": fi}

### Dividends History Fetcher
This tool retrieves the most recent dividend events for a given ticker via `yfinance`. It returns up to a given number of past dividends (5), each with a date and dividend amount. If the ticker has no dividends or the data is empty, it returns an empty list.

In [16]:
@tool("Fetch dividends history (compact)")
def yf_dividends(ticker: str, limit: int = 5) -> Dict[str, Any]:
    """Returns dividends history (compact)"""
    
    tk = yf.Ticker(ticker)
    s = tk.dividends
    
    if s is None or s.empty: # if no dividends 
        return {"ticker": ticker, "dividends": []}
        
    df = s.reset_index().rename(columns={"Date": "date", "Dividends": "dividend"})
    return {"ticker": ticker, "dividends": df.tail(limit).to_dict(orient="records")}

### Earnings Calendar & Next Events Fetcher
This tool attempts to fetch upcoming or recent calendar events (e.g. earnings, dividends) for a ticker via `yfinance.Ticker.calendar`. It returns a structured list of event names and values if available; otherwise it returns an empty list.

In [18]:
@tool("Fetch earnings calendar / next events")
def yf_calendar(ticker: str) -> Dict[str, Any]:
    """Returns earnings calendar / next events"""
    
    tk = yf.Ticker(ticker)
    
    try:
        cal = tk.calendar
        if cal is None or cal.empty: # if no events
            return {"ticker": ticker, "calendar": []}
            
        df = cal.reset_index().rename(columns={"index": "event", 0: "value"})
        return {"ticker": ticker, "calendar": df.to_dict(orient="records")}
    except Exception: # if no success with yfinance endpoint
        return {"ticker": ticker, "calendar": []}

### Return Past One-Sentence Takeaways 
The load_kb_lessons_tool is used to load previously saved lessons associated with a specific stock ticker. These one-sentence takeaways capture key insights or lessons learned from prior reviews, enabling the system to build on past analysis and maintain consistency across research cycles.

In [20]:
@tool("Return past one-sentence takeaways (lessons) for a ticker, one per line.")
def load_kb_lessons_tool(ticker: str) -> str:
    """Return past one-sentence takeaways (lessons) for a ticker, one per line."""
    lessons = kb_load_lessons(ticker)
    if not lessons:
        return "(no prior lessons)"
    return "\n".join(f"- {i+1}. {row.get('lesson', '')}" for i, row in enumerate(lessons))


### Append a New One-Sentence Lesson
The save_kb_lesson_tool is responsible for adding new lessons to the knowledge base. Each entry is stored in a JSONL file with a timestamp and the associated stock ticker, ensuring that every new insight is chronologically tracked and available for future reference.

In [22]:
@tool("Append a new one-sentence lesson for a ticker to the KB.")
def save_kb_lesson_tool(ticker: str, lesson: str) -> str:
    """Append a new one-sentence lesson for a ticker to the KB."""
    kb_append_lesson(ticker, {"lesson": lesson})
    return f"Saved lesson for {ticker}: {lesson}"

## Agents
An agent is an autonomous AI entity defined by a role, goal, a backstory (context) and tools. It reasons, plans, and acts on tasks using its tools, and can collaborate within a crew.  

### Agents in the Crew:
- Planner: Designs a structured, step-by-step plan (5–7 bullets) for how to research a given ticker using the Yahoo Finance tools.
- Researcher: Executes the plan by calling the Yahoo Finance tools (prices, fundamentals, dividends, calendar) and synthesizes findings, including a Buy/Sell/Hold recommendation and compact data summary.
- Reviewer: Reviews the Researcher's draft output and generates feedback and improvement suggestions (a review plan) for the next run.
- Optimizer: Takes the Researcher's work plus Reviewer feedback; modifies and refines the draft accordingly, and stores lessons learned to improve future runs.

### Planner Agent
This agent's role is to design a compact, tactical research plan for a given ticker. It does not fetch data itself, instead it creates a 5-7 step sequence that a Researcher agent can follow using only the Yahoo Finance tools (prices, fundamentals, dividends, calendar).  

In [25]:
planner = Agent(
    role="Planner", # role
    goal=( # goal prompt
        "Design a concise, step-by-step plan (5–7 bullets) for researching {ticker} "
        "using only the Yahoo Finance tools available to the Researcher."
    ),
    backstory=( # context
        "You are a methodical planning specialist. Your job is to outline an efficient "
        "sequence of steps that the Researcher can follow to gather price context, "
        "key fundamentals, dividends, upcoming events, and any red-flag checks."
    ),
    tools=[], # no tools used
    allow_delegation=False, # no delegation to other agents
    memory=False, # no retain of memory (maybe?)
    verbose=True, # enable logging of internal reasoning
    # llm=openai_llm, # the LLM to be used; gpt-4o-mini is too small for this task
    max_iter=2, # number of reasoning iterations
    max_rpm=30, # max request per mins
)

### Researcher Agent
This agent uses Yahoo Finance tools (prices, fundamentals, dividends, calendar) to generate a compact, data-driven analysis of a given stock. Its output includes a ~200-word synthesis, a Buy/Sell/Hold recommendation, and a concise data snapshot.  

In [27]:
researcher = Agent(
    role="Researcher",
    goal=(
        "Use Yahoo Finance tools to gather a concise, data-backed snapshot and "
        "incorporate prior one-sentence lessons to avoid repeating past mistakes. "
        "Produce a ~200-word synthesis, a Buy/Sell/Hold recommendation, and a compact data summary."
    ),
    backstory=(
        "You are a pragmatic equity researcher. You rely only on the provided tools, "
        "call out any missing/uncertain data, and fold in historical lessons for the ticker."
    ),
    tools=[yf_prices, yf_fundamentals, yf_dividends, yf_calendar, load_kb_lessons_tool], 
    allow_delegation=False,
    memory=False,
    verbose=True,
    llm=openai_llm,
    max_iter=3,
    max_rpm=30,
)


### Reviewer Agent

This is a quality-assurance agent that rigorously critiques the Researcher's draft. It assigns a grade (1–5), provides a concise rationale, and produces a concrete improvement plan (3–6 bullets). It also captures a single lesson sentence for future use.  

In [29]:
#Agent
reviewer = Agent(
    role="Reviewer",
    goal=(
        "Judge the Researcher’s draft strictly, assign a single 1–5 rank, "
        "provide rationale, and produce a concrete improvement plan for the Optimizer. "
        "Store a concise lesson."
    ),
    backstory=(
        "A meticulous investment research QA who grades with a tough rubric "
        "and writes crisp, actionable feedback."
    ),
    tools=[load_kb_lessons_tool, save_kb_lesson_tool,
          yf_prices, yf_fundamentals, yf_dividends, yf_calendar], # same tools as the Researcher
    allow_delegation=False,
    verbose=True,
    llm=claude_llm,
)

## Tasks
A Task is an assignment that an Agent must complete. It contains everything needed to perform such as the description telling the agent what to do, the agent responsible for executing it, expected outputs/format, context and tools that the agent can use during task execution. 

### Planning Task 
This  Task directs the Planner Agent to produce a structured 5–7 step **RESEARCH_PLAN** for the Researcher agent, using only the allowed Yahoo Finance tools (`yf_prices`, `yf_fundamentals`, `yf_dividends`, `yf_calendar`).  

In [32]:
PLANNING_PROMPT = """You are the Equity Research Planner.

Ticker: {ticker}

INSTRUCTIONS:
- Draft a 5–7 step RESEARCH_PLAN for the Researcher that uses ONLY these tools:
  - yf_prices(ticker, period='90d', interval='1d')
  - yf_fundamentals(ticker)
  - yf_dividends(ticker, limit=5)
  - yf_calendar(ticker)
- Order steps logically (price context → fundamentals → dividends → calendar → red flags).
- Be specific about periods/parameters the Researcher should try.
- If data could be missing, note a fallback (e.g., try a longer period).

OUTPUT:
**RESEARCH_PLAN** — numbered bullets
"""

# creates a CrewAI task
plan_research = Task(
    description=PLANNING_PROMPT, # prompt with the instructions for the task
    agent=planner, # the agent responsable for this task (Planner)
    expected_output="RESEARCH_PLAN" # the expected output
)

### Research Task: 
This Task directs the Researcher to produce a data-driven SYNTHESIS, a Buy/Sell/Hold recommendation with rationale, and a compact DATA SNAPSHOT. It instructs the agent to consume a prior RESEARCH_PLAN or draft one itself, then use the Yahoo Finance tools (`yf_prices`, `yf_fundamentals`, `yf_dividends`, `yf_calendar`) to pull data and structure the output accordingly.

In [34]:
RESEARCH_PROMPT = RESEARCH_PROMPT = """You are the Equity Researcher.

Ticker: {ticker}

TOOLS AVAILABLE:
- yf_prices(ticker, period='90d', interval='1d'): recent OHLCV
- yf_fundamentals(ticker): compact fundamentals snapshot
- yf_dividends(ticker, limit=5): recent dividends
- yf_calendar(ticker): next/recent calendar entries (e.g., earnings)
- load_kb_lessons_tool(ticker, n=5): retrieve up to n prior one-sentence lessons for this ticker

INSTRUCTIONS:
0) Call load_kb_lessons_tool(ticker={ticker}, n=5). Identify the 1–2 most relevant lessons and explicitly apply them to guide your analysis (e.g., ensure peer context if a past lesson flagged it).
1) Call the tools you need to understand {ticker}'s recent behavior and snapshot.
2) Summarize price context, key fundamentals (best-effort), any dividends, and upcoming events if present.
3) If a tool returns empty/missing fields, acknowledge it and suggest a reasonable fallback (e.g., longer period).
4) Produce a Buy/Sell/Hold recommendation.

OUTPUT:
**SYNTHESIS (~150–250 words)** — weave in how prior lessons were applied (e.g., “Applied prior lesson: provide peer P/E context”).
**RECOMMENDATION** — a one-word classification (BUY/SELL/HOLD) plus one short sentence of rationale, and include "recommendation" inside the JSON snapshot.
**DATA SNAPSHOT** — compact JSON that includes:
{
  "ticker": "{ticker}",
  "recommendation": "<BUY|SELL|HOLD>",
  "signals": { ... },           # key numbers you surfaced
  "events": { ... },            # calendar if any
  "lessons_used": ["<lesson 1>", "<lesson 2>"]  # exactly the lessons you actually applied
}
"""

research_ticker = Task(
    description=RESEARCH_PROMPT,
    agent=researcher, # executed by Researcher Agent
    expected_output="SYNTHESIS, RECOMMENDATION, DATA SNAPSHOT",
    context=[plan_research] # passes the Planner's output to the Researcher as context
)

### Review Task: 
This task takes the Researcher’s draft output (via `context=[research_ticker]`) and critiques it.  

The Reviewer:
- Assigns a grade (1–5),
- Writes a rationale (2–5 sentences),
- Crafts a concrete improvement plan (3–6 bullets),
- Produces a single lesson sentence to store for future runs.

The Reviewer task runs after the Researcher in the sequential Crew, ensuring it always has the Researcher’s output to review.


In [36]:
# UPDATE: Reuses Researcher tools now and added a bit more a rubric to the reviews. Added Markdown format. Added IMprovement plan "command-like" output

In [37]:
REVIEW_PROMPT = """You are reviewing an investment research draft.

Inputs:
- Ticker: {ticker}

Context (tools available):
- You may load prior one-sentence takeaways (lessons) for this ticker.
- (Optional, if enabled by the system) You may call the same Yahoo Finance tools available to the Researcher to quickly verify obvious gaps or claims.

Your job (be strict, brief, and actionable):
1) Assign a 1–5 integer grade (1 = unacceptable, 3 = adequate/minimum viable, 5 = excellent),
   scored on: Correctness (40%), Completeness (35%), Clarity/Structure (25%).
2) Write a short rationale (2–5 crisp sentences) that cites the specific missing/weak pieces (e.g., “no P/E context vs peers”).
3) Produce an **improvement plan** targeted at an execution agent named “Optimizer”.
   - 3–8 concrete steps.
   - Each step must be a command-like instruction the Optimizer can act on immediately.
   - Prefer steps that reference available tools (prices, fundamentals, dividends, calendar, lessons).
   - If data is missing, instruct the Optimizer exactly what to fetch and how to incorporate it.
4) Write exactly one single-sentence “lesson” that would have prevented most issues in this draft.

Output: STRICT JSON only, matching this schema exactly:
{
  "grade": <integer 1-5>,
  "rationale": "<2-5 sentences>",
  "plan": [
    "Optimizer: <do X using <tool> …>",
    "Optimizer: <do Y …>",
    "... (3-8 items total)"
  ],
  "lesson": "<one sentence>"
}

Constraints:
- No text outside the JSON.
- Be specific. Avoid vague comments like “add more analysis”.
- If the Researcher already did a step well, do not duplicate it in the plan; focus on deltas.
- If you used any tool checks yourself, they should influence the critique, but do NOT print tool logs.
"""

review_task = Task(
    description=REVIEW_PROMPT,
    agent=reviewer,
    expected_output="Strict JSON with keys grade, rationale, plan, lesson.",
    output_file=str(OUT_DIR / "reviewer_output.json"),
    context=[research_ticker]  # added Researcher's task output
)

### Archive Lesson Task:
This task directs the Reviewer to archive the one-sentence lesson from its own review output. It extracts the lesson, optionally checks for duplicates with load_kb_lessons_tool, saves it using save_kb_lesson_tool, and outputs a JSON confirmation indicating whether the lesson was successfully stored.

In [38]:
ARCHIVE_LESSON_PROMPT = """You are the same Reviewer, now acting as a lesson archivist.

Inputs:
- Ticker: {ticker}

Context:
- You will receive the Reviewer’s STRICT JSON (keys: grade, rationale, plan, lesson) as context.

Your job:
1) Parse the Reviewer JSON from context and extract the single-sentence "lesson".
2) (Optional) Check recent lessons for this ticker (via load_kb_lessons_tool) to avoid exact duplicates.
3) CALL save_kb_lesson_tool(ticker={ticker}, lesson=<lesson>).
4) Output STRICT JSON ONLY:

{
  "ticker": "<TICKER>",
  "saved": true,
  "reason": "ok",
  "lesson": "<the one-sentence lesson>"
}

If duplicate or missing lesson:
- Set "saved": false and give a short reason (e.g., "duplicate", "lesson missing").
Constraints:
- No text outside the JSON.
"""

archive_lesson_task = Task(
    description=ARCHIVE_LESSON_PROMPT,
    agent=reviewer,  # same agent
    expected_output='Strict JSON with keys: ticker, saved, reason, lesson.',
    output_file=str(OUT_DIR / "archivist_output.json"),
    context=[review_task],  # feeds in the Reviewer JSON
)


## Crew Setup
Here we create a Crew that coordinates the agents and the tasks.

When `crew.kickoff()` is called, the Planner will produce a plan, then the Researcher uses that plan to drive its analysis....  


In [40]:
crew = Crew(
    agents=[planner, researcher, reviewer], # list agents        
    tasks=[plan_research, research_ticker, review_task, archive_lesson_task], #  list of tasks to execute in order
    process=Process.sequential, # task to run one after another
    verbose=True # enable logging
)

crew

Crew(id=49c87bb9-f158-4686-bc88-bbb7f7353e33, process=Process.sequential, number_of_agents=3, number_of_tasks=4)

In [41]:
# execute the crew's tasks in sequence using an input
result = crew.kickoff(inputs={"ticker": "AAPL"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 49c87bb9-f158-4686-bc88-bbb7f7353e33                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planner                                                                                                 │
│                                                                                                                 │
│  Task: You are the Equity Research Planner.                                                                     │
│                                                                                                                 │
│  Ticker: AAPL                                                                                                   │
│                                                                                                                 │
│  INSTRUCTIONS:                                                                                                  │
│  - Draft a 5–7 step RESEARCH_PLAN for the Researcher that uses ONLY these tools:                                │
│    - yf_prices(ticker, period='90d', interval='1d')                                                             │
│    - yf_fundamentals(ticker)                                                                                    │
│    - yf_dividends(ticker, limit=5)                                                                              │
│    - yf_calendar(ticker)                                                                                        │
│  - Order steps logically (price context → fundamentals → dividends → calendar → red flags).                     │
│  - Be specific about periods/parameters the Researcher should try.                                              │
│  - If data could be missing, note a fallback (e.g., try a longer period).                                       │
│                                                                                                                 │
│  OUTPUT:                                                                                                        │
│  **RESEARCH_PLAN** — numbered bullets                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Planner                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  **RESEARCH_PLAN**                                                                                              │
│  1. Use the `yf_prices('AAPL', period='90d', interval='1d')` tool to gather price data for Apple Inc. over the  │
│  last 90 days, which will help identify any price trends and volatility. If data is missing, try extending the  │
│  period to 180 days.                                                                                            │
│                                                                                                                 │
│  2. Access the `yf_fundamentals('AAPL')` tool to retrieve key fundamentals. Focus on metrics such as P/E        │
│  ratio, EPS, revenue growth, and debt-to-equity ratio for a comprehensive understanding of the company's        │
│  financial health.                                                                                              │
│                                                                                                                 │
│  3. Utilize the `yf_dividends('AAPL', limit=5)` tool to examine the last five dividend payments. Note the       │
│  dividend yield, payment dates, and any changes in dividend amounts to assess the company’s commitment to       │
│  returning value to shareholders.                                                                               │
│                                                                                                                 │
│  4. Check the `yf_calendar('AAPL')` tool for upcoming earnings reports and significant corporate events. Pay    │
│  attention to the dates and details as these can influence stock performance.                                   │
│                                                                                                                 │
│  5. Conduct red-flag checks by comparing the latest results from the fundamentals and dividends with the        │
│  historical data you gather in the previous steps. Identify any discrepancies or concerns, such as declining    │
│  earnings against increasing dividends, which may suggest risks.                                                │
│                                                                                                                 │
│  6. Compile notes and observations from each step, looking for correlations and potential factors driving       │
│  AAPL's performance. This summary will aid in creating a well-informed investment perspective.                  │
│                                                                                                                 │
│  7. Finally, document any broader market conditions or news that could impact AAPL during your research period  │
│  to contextualize the findings. This can include interest rates, technological trends, or competitive           │
│  landscape changes.                                                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 82cc6551-98b6-43b5-988b-31c9c7ceb3e2                                                                     │
│  Agent: Planner                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Saved lesson for AAPL: Always include comprehensive financial analysis, competitive positioning, and specific  │
│  price targets to provide a well-rounded and actionable investment recommendation.                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Reviewer                                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "ticker": "AAPL",                                                                                            │
│    "saved": true,                                                                                               │
│    "reason": "ok",                                                                                              │
│    "lesson": "Always include comprehensive financial analysis, competitive positioning, and specific price      │
│  targets to provide a well-rounded and actionable investment recommendation."                                   │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0b56ea52-fd56-4aa6-9afa-c9e4337757b4                                                                     │
│  Agent: Reviewer                                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 49c87bb9-f158-4686-bc88-bbb7f7353e33                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "ticker": "AAPL",                                                                                            │
│    "saved": true,                                                                                               │
│    "reason": "ok",                                                                                              │
│    "lesson": "Always include comprehensive financial analysis, competitive positioning, and specific price      │
│  targets to provide a well-rounded and actionable investment recommendation."                                   │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [42]:
# print each task result 
display(Markdown("# Planner Agent Output"))
display(Markdown(plan_research.output.raw))

# Planner Agent Output

**RESEARCH_PLAN**  
1. Use the `yf_prices('AAPL', period='90d', interval='1d')` tool to gather price data for Apple Inc. over the last 90 days, which will help identify any price trends and volatility. If data is missing, try extending the period to 180 days.
  
2. Access the `yf_fundamentals('AAPL')` tool to retrieve key fundamentals. Focus on metrics such as P/E ratio, EPS, revenue growth, and debt-to-equity ratio for a comprehensive understanding of the company's financial health.
  
3. Utilize the `yf_dividends('AAPL', limit=5)` tool to examine the last five dividend payments. Note the dividend yield, payment dates, and any changes in dividend amounts to assess the company’s commitment to returning value to shareholders.
  
4. Check the `yf_calendar('AAPL')` tool for upcoming earnings reports and significant corporate events. Pay attention to the dates and details as these can influence stock performance.
  
5. Conduct red-flag checks by comparing the latest results from the fundamentals and dividends with the historical data you gather in the previous steps. Identify any discrepancies or concerns, such as declining earnings against increasing dividends, which may suggest risks.
  
6. Compile notes and observations from each step, looking for correlations and potential factors driving AAPL's performance. This summary will aid in creating a well-informed investment perspective.
  
7. Finally, document any broader market conditions or news that could impact AAPL during your research period to contextualize the findings. This can include interest rates, technological trends, or competitive landscape changes.

In [43]:
display(Markdown("# Researcher Agent Output"))
display(Markdown(research_ticker.output.raw))

# Researcher Agent Output

**SYNTHESIS**  
Apple Inc. (AAPL) presents a stable and appealing investment proposition. Over the past 90 days, its stock experienced moderate volatility but remained within a high proximity to its 52-week peak, reflecting strong investor confidence. The company holds a significant market cap exceeding $3.8 trillion, situating it as a leading player in the tech sector. Dividends have shown consistency with a slight increment, reflecting Apple's commitment to returning shareholder value—a positive signal for income-focused investors. However, the absence of forthcoming earnings announcements suggests potential uncertainties for short-term investors relying on explicit guidance. Applied prior lessons encouraged a balance of financial ratios with an emphasis on dividends and price stability over time to forecast risk and optimism in Apple’s performance effectively.

**RECOMMENDATION**  
HOLD - AAPL's strong fundamentals and appeal as a stable, dividend-paying giant encourage holding for long-term gains amidst short-term event uncertainty.

**DATA SNAPSHOT**  
```json
{
  "ticker": "AAPL",
  "recommendation": "HOLD",
  "signals": {
    "market_cap": 3809379745331.421,
    "year_high": 260.10,
    "year_low": 169.21,
    "last_price": 256.69,
    "recent_dividends": [0.25, 0.25, 0.25, 0.26, 0.26]
  },
  "events": {},
  "lessons_used": [
    "Always include comparative analysis, specific price levels, and a balanced view of risks and opportunities to provide a well-rounded investment recommendation.",
    "Always include key financial ratios, technical indicators, and competitive analysis to provide a comprehensive and well-supported investment recommendation."
  ]
}
```

In [74]:
display(Markdown("# Reviewer Agent Output"))
display(Markdown(review_task.output.raw))
display(Markdown(archive_lesson_task.output.raw))

# Reviewer Agent Output

{
  "grade": 3,
  "rationale": "The draft provides a basic overview of AAPL's stock performance and dividends, but lacks depth in several key areas. It fails to include specific financial ratios, competitive analysis, and a more detailed discussion of risks and opportunities. The recommendation lacks concrete price targets or entry/exit points, which would be valuable for investors.",
  "plan": [
    "Optimizer: Use the 'Fetch a fundamentals snapshot from Yahoo Finance' tool to gather key financial ratios (P/E, P/B, ROE) and include them in the analysis.",
    "Optimizer: Analyze the recent OHLCV data from the 'Fetch recent OHLCV prices from Yahoo Finance' tool to identify specific support and resistance levels.",
    "Optimizer: Research and include a brief competitive analysis comparing AAPL to at least two major competitors in the tech sector.",
    "Optimizer: Use the 'Fetch earnings calendar / next events' tool to check for any upcoming events that could impact the stock price.",
    "Optimizer: Provide a more detailed risk assessment, including potential challenges in the smartphone market and global economic factors.",
    "Optimizer: Set specific price targets for entry, exit, and stop-loss points based on the technical analysis of recent price data.",
    "Optimizer: Expand on the dividend analysis by calculating the dividend yield and comparing it to industry averages."
  ],
  "lesson": "Always include comprehensive financial analysis, competitive positioning, and specific price targets to provide a well-rounded and actionable investment recommendation."
}

{
  "ticker": "AAPL",
  "saved": true,
  "reason": "ok",
  "lesson": "Always include comprehensive financial analysis, competitive positioning, and specific price targets to provide a well-rounded and actionable investment recommendation."
}